# Handling Python Enums

Very often we use Enums in our models in order to restrict fields to only certain values.

For example, we might have a field that should be a list of pre-defined colors.

In our Python code, we don't want to use just string literals - that's prone to error (typos), so we use enumerations.

In [1]:
from enum import Enum

In [2]:
class Color(Enum):
    red = "Red"
    green = "Green"
    blue = "Blue"
    orange = "Orange"
    yellow = "Yellow"
    cyan = "Cyan"
    white = "White"
    black = "Black"

In our Python code we then simply use these colors by using their enumeration values:

In [3]:
Color.red

<Color.red: 'Red'>

We can recover the associated value:

In [4]:
Color.orange.value

'Orange'

Suppose we have an API endpoint where the request body requires one of these colors to be specified.

We would like to use that enumeration in our model, to impose a constraint as to what values can be passed in, **and** map it back to our enumeration when the data gets deserialized.

We can simply specify the enumeration as the field type:

In [5]:
from pydantic import BaseModel, ConfigDict, ValidationError

In [6]:
class Model(BaseModel):
    color: Color

Now let's try deserializing using Python objects:

In [7]:
Model(color=Color.red)

Model(color=<Color.red: 'Red'>)

And what about deserializing a JSON string?

In [8]:
data = """
{
    "color": "Red"
}
"""

Model.model_validate_json(data)

Model(color=<Color.red: 'Red'>)

And if we choose a value that is not in our enumeration:

In [9]:
data = """
{
    "color": "Magenta"
}
"""

try:
    Model.model_validate_json(data)
except ValidationError as ex:
    print(ex)

1 validation error for Model
color
  Input should be 'Red', 'Green', 'Blue', 'Orange', 'Yellow', 'Cyan', 'White' or 'Black' [type=enum, input_value='Magenta', input_type=str]


What about serializing the model? If we serialize to a Python dictionary, we would expect the enumeration type to be kept in the serializtion:

In [10]:
data = """
{
    "color": "Red"
}
"""

m = Model.model_validate_json(data)

In [11]:
m.model_dump()

{'color': <Color.red: 'Red'>}

And if we serialize to JSON:

In [12]:
m.model_dump_json()

'{"color":"Red"}'

So, we see that when we deserialize the data, the field is populated with a enum member.

But what if we actually just want the value of the enum, instead of the enum member itself - i.e. we want to use an enum in our model to just limit the values that will be accepted, and keep the values as they are, not as enum members?

We can do this by setting a configuration on our model: `use_enum_values`, which by default is `False`.

In [13]:
class Model(BaseModel):
    model_config = ConfigDict(use_enum_values=True)

    color: Color

In [14]:
m = Model(color=Color.cyan)

In [15]:
m.color

'Cyan'

In [16]:
type(m.color)

str

As you can see, our field is now a string, not an enumeration member.

And of course, serializing to a Python dict will also serialize the field as a string:

In [17]:
m.model_dump()

{'color': 'Cyan'}

But, our validation still holds:

In [18]:
data = """
{
    "color": "Magenta"
}
"""

try:
    Model.model_validate_json(data)
except ValidationError as ex:
    print(ex)

1 validation error for Model
color
  Input should be 'Red', 'Green', 'Blue', 'Orange', 'Yellow', 'Cyan', 'White' or 'Black' [type=enum, input_value='Magenta', input_type=str]


There is a small caveat though - and that's when we set a default value.

Let's take a look:

In [19]:
class Model(BaseModel):
    model_config = ConfigDict(use_enum_values=True)

    color: Color = Color.red

Now, let's create a model instance with the default - it **should** be a string with value `"Red"`:

In [20]:
m = Model()
m.color, type(m.color)

(<Color.red: 'Red'>, <enum 'Color'>)

And that's not at all what we expected!

If we pass a value when deseralizing things work as expected:

In [21]:
m = Model(color=Color.red)
m.color, type(m.color)

('Red', str)

Even with JSON deserialization:

In [22]:
data = """{}"""

m = Model.model_validate_json(data)
m.color, type(m.color)

(<Color.red: 'Red'>, <enum 'Color'>)

And the problem is one we've seen before - default values do not get validated - they are used as-is (by default).

So, we can fix this behavior by setting a model level configuration to validate defaults (later we'll see that we can do this at the field level, rather than at the model level, which applies to every field).

In [23]:
class Model(BaseModel):
    model_config = ConfigDict(use_enum_values=True, validate_default=True)

    color: Color = Color.red

In [24]:
m = Model()
m.color, type(m.color)

('Red', str)

As you can see, our default value was deserialized to a string.

So, just something to keep in mind when you choose to use `use_enum_values`.

Is there any reason to use that option?

Not really, the reasons, as far as my experience goes, is very limited. It usually has to do with trying to serialize the result of a `model_dump()` to JSON ourselves - since we would not be utilizing Pydantic to do that serialization, having an Enum member in that dictionary would need us to handle it specially, since enums are not serializable by default when using the `json.dumps()` function.

Apart from that, and there are ways to deal with that problem, I do not really see a use for that option. However, it is available if you need it.